In [0]:
# colab
# !pip3 install arviz

In [0]:
import os 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

import pystan
from criterion import Criterion

import arviz
import pickle

from IPython.display import display


plt.style.use("ggplot")

In [0]:
# modelを記述した読み込みたいstanファイル
read_stanmodel = "model7-3.stan"
# モデルをセーブしておくpickleファイル
read_file = "model7-3.pkl"

stan_path = Path(os.getcwd(),"model", "stanmodel", read_stanmodel)
pickle_path = Path(os.getcwd(),"model", "model_pkl", read_file)

def read_stanmodel(stan_path, pickle_path):
        
    try:
        with open(pickle_path, "rb") as f:
            print("loading...", pickle_path)
            stanmodel = pickle.load(f)

    except FileNotFoundError:
        print("save path to stan file is ", stan_path)
        stanmodel = pystan.StanModel(
            file = str(stan_path),
        )
        with open(pickle_path, "wb") as f:
            pickle.dump(stanmodel, f)
            print("saving finished...")
    return stanmodel

In [0]:
stanmodel = read_stanmodel(stan_path, pickle_path)

In [0]:
data = pd.read_csv(Path(os.getcwd(), "input", "data-aircon.txt"))

In [0]:
display(data.head())
display(data.describe())

In [0]:
plt.scatter(data["X"], data["Y"])
plt.show()

In [0]:
N_new = 60
X_new = np.linspace(-3, 32, N_new)
standata = {
    "N":len(data),
    "X":data["X"],
    "Y":data["Y"],
    "N_new":N_new,
    "X_new":X_new
}

In [0]:
fit = stanmodel.sampling(
    data=standata,
    iter=3000,
    warmup=300,
    chains=4,
)

In [0]:
print(fit)

In [0]:
arviz.plot_trace(fit);

In [0]:
ms = fit.extract()

In [0]:
ms.keys()

In [0]:
inference_data = pd.DataFrame(data={"X":X_new, "y_new": np.mean(ms["y_new"], axis=0)})

In [0]:
plt.scatter(inference_data["X"], inference_data["y_new"])

In [0]:
mean = np.mean(ms["y_new"], axis=0)
std = np.std(ms["y_new"], axis=0)
# print(f"mean: {mean} \n std: {std}")
upper = mean + 1.96 * std
lower = mean - 1.96 * std

In [0]:
plt.plot(inference_data["X"], inference_data["y_new"])
plt.fill_between(inference_data["X"], upper, lower, alpha=0.4)
plt.show()

In [0]:
def plot_inference_data(inference_x : pd.DataFrame, 
                        inference_y : pd.DataFrame, 
                        interval_coef=1.96, save=False, out_dir=None, png_name=None):
    
    mean = np.mean(inference_y, axis=0)
    std = np.std(inference_y,  axis=0)
    # print(f"mean: {mean.shape} \n std: {std.shape}")
    upper: np.array = mean + interval_coef * std
    lower: np.array = mean - interval_coef * std
    # drawing png
    plt.plot(inference_x, mean, label="mean")
    plt.fill_between(inference_x, upper, lower, alpha=0.4)
    plt.title("inference data and prediction")
    # save png 
    if save:
        if out_dir is None:
            Path.mkdir(os.getcwd(), "save_stanpng")
            outdir = str(Path(os.getcwd(), "save_stanpng"))
        plt.savefig(Path(out_dir, "inference_data_"+png_name))
        plt.close()
    plt.show()


In [0]:
plot_inference_data(inference_data["X"], ms["y_new"])

In [0]:
inference_data["y_new"]

In [0]:
ms["y_new"]